In [ ]:
!git clone https://github.com/nkt780426/Bachelor-s-Project.git

In [ ]:
!mv Bachelor-s-Project/going_modular .

In [1]:
!ls -la

total 12
drwxr-xr-x 3 root root 4096 Dec 20 01:50 .
drwxr-xr-x 5 root root 4096 Dec 20 01:50 ..
drwxr-xr-x 2 root root 4096 Dec 20 01:50 .virtual_documents


In [ ]:
# !rm -rf checkpoint/

In [ ]:
# !rm -rf going_modular

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import torch
from torchvision import transforms
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts
import albumentations as A


from going_modular.model.TripletFaceRecognition import EmbeddingNet, TripletNet
from going_modular.dataloader.triplet import TripletDataset
from going_modular.loss.TripletLoss import TripletLoss
from going_modular.train_eval.triplet.train import fit
from going_modular.dataloader.triplet import CustomExrDataset
from going_modular.utils.MultiMetricEarlyStopping import MultiMetricEarlyStopping
from going_modular.utils.ModelCheckPoint import ModelCheckpoint
from going_modular.utils.transforms import RandomResizedCropRect, GaussianNoise

import os

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

# Đặt seed toàn cục
seed = 42
torch.manual_seed(seed)

CONFIGURATION = {
    # Thư mục
    'type': 'albedo',
    'train_dir': '/kaggle/input/hoangvn-3dfacerecognition/Albedo/train',
    'test_dir': '/kaggle/input/hoangvn-3dfacerecognition/Albedo/test',
    
    # Cấu hình train
    'epochs': 500,
    'num_workers': 4,
    'batch_size': 16,
    'image_size': 256,
    'embedding_size': 512,
    
    'start_lr': 1e-4,
    'weight_decay': 5e-4,
    'momentum': 0.9,
    'alpha': 0.9,
    
    # triplet
    'margin': 1.,
}

train_transform = A.Compose([
    RandomResizedCropRect(256),
    GaussianNoise(),
])

test_transform = A.Compose([
    A.Resize(height=CONFIGURATION['image_size'], width=CONFIGURATION['image_size'])
])

triplet_train_dataset = TripletDataset(data_dir=CONFIGURATION['train_dir'], transform=train_transform, train=True)
triplet_test_dataset = TripletDataset(data_dir=CONFIGURATION['train_dir'], transform=test_transform, train=False)

triplet_train_loader = DataLoader(
    triplet_train_dataset, 
    batch_size=CONFIGURATION['batch_size'], 
    shuffle=True,
    num_workers=4,
    pin_memory=True
)

triplet_test_loader = DataLoader(
    triplet_test_dataset, 
    batch_size=CONFIGURATION['batch_size'], 
    shuffle=False,
    num_workers=4,
    pin_memory=True
)

roc_train_dataset = CustomExrDataset(CONFIGURATION['train_dir'], transform=train_transform, type=CONFIGURATION['type'])
roc_train_loader = DataLoader(
    roc_train_dataset,
    num_workers=4,
    batch_size=CONFIGURATION['batch_size'],
)

roc_test_dataset = CustomExrDataset(CONFIGURATION['test_dir'], transform=test_transform, type=CONFIGURATION['type'])
roc_test_loader = DataLoader(
    roc_test_dataset,
    num_workers=4,
    batch_size=CONFIGURATION['batch_size'],
)

embedding_net = EmbeddingNet(len_embedding=CONFIGURATION['embedding_size']).to(device)
model = TripletNet(embedding_net).to(device)
criterion = TripletLoss(CONFIGURATION['margin'])
optimizer = Adam(model.parameters(), lr=CONFIGURATION['start_lr'])
scheduler = CosineAnnealingWarmRestarts(optimizer, T_0=50, T_mult=1, eta_min=1e-6)

earlystop_dir = os.path.abspath('checkpoint/triplet/' + CONFIGURATION['type'] + '/models')
early_min_stopping = MultiMetricEarlyStopping(
    monitor_keys=['test_loss'],
    patience=50,
    mode='min',
    verbose=0,
    save_dir=earlystop_dir,
    start_from_epoch=10
)      

early_max_stopping = MultiMetricEarlyStopping(
    monitor_keys=['cosine_auc', 'cosine_acc', 'euclidean_auc', 'euclidean_acc'],
    patience=50,
    mode='max',
    verbose=1,
    save_dir=earlystop_dir,
    start_from_epoch=20
)      

checkpoint_path = os.path.abspath('checkpoint/triplet/' + CONFIGURATION['type'] + '/models/checkpoint.pth')
modle_checkpoint = ModelCheckpoint(filepath=checkpoint_path, verbose=1)

In [ ]:
fit(
    conf=CONFIGURATION,
    start_epoch=0,
    model=model,
    triplet_train_loader=triplet_train_loader, 
    triplet_test_loader=triplet_test_loader, 
    criterion=criterion,
    optimizer=optimizer, 
    scheduler=scheduler, 
    epochs=CONFIGURATION['epochs'], 
    device=device, 
    roc_train_loader=roc_train_loader, 
    roc_test_loader=roc_test_loader,
    early_max_stopping=early_max_stopping,
    early_min_stopping=early_min_stopping,
    model_checkpoint=modle_checkpoint
)